**Gesture Recognition**

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
import numpy as np
import os
import imageio.v2 as imageio
from imageio.v2 import imread
from skimage.transform import resize
import datetime
import os
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

from keras.applications import mobilenet

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Mounting Data (Running on Google Colab as Jarvis Ai labs workspace is having py environment issues. Not able to import necessary modules and libraries)

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train_doc = np.random.permutation(open('/content/gdrive/My Drive/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/gdrive/My Drive/Project_data/val.csv').readlines())
batch_size = 64

**GEnerator**

The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with img_idx, y,z and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size, img_idx, image_height, image_width):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # img_idx = [1,4,7,10,13,16,19,22,25,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size) # calculate the number of batches
        # left over batches which should be handled separately
        leftover_batches = len(t) - num_batches * batch_size

        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size, len(img_idx), image_height, image_width, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image, (image_height, image_width))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]) - 104
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]) - 117
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]) - 123

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        # write the code for the remaining data points which are left after full batches
        if leftover_batches != 0:
            for batch in range(num_batches):
                # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_data = np.zeros((batch_size,len(img_idx),image_height, image_width,3))
                # batch_labels is the one hot representation of the output: 10 videos with 5 columns as classes
                batch_labels = np.zeros((batch_size,5))
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path +'/'+t[batch * batch_size + folder].split(';')[0])
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                        image = imageio.imread(source_path +'/'+t[batch * batch_size + folder].split(';')[0] +'/'+imgs[item]).astype(np.float32)
                        image = resize(image, (image_height,image_width))

                        batch_data[folder,idx,:,:,0] = (image[:,:,0]) - 104
                        batch_data[folder,idx,:,:,1] = (image[:,:,1]) - 117
                        batch_data[folder,idx,:,:,2] = (image[:,:,2]) - 123

                    #Fill the one hot encoding stuff where we maintain the label
                    batch_labels[folder, int(t[batch * batch_size + folder].split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/gdrive/My Drive/Project_data/train'
val_path = '/content/gdrive/My Drive/Project_data/val'
model_path_prefix = '/content/gdrive/My Drive/Models/'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


In [8]:
img_idx = list(range(1,30,3))
image_height = 50
image_width = 50
batch_size = 1
test_gen = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
d = next(test_gen)[0]

Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 1


Model 1 - 50x50 - No Dropout - Layers [16 > 32 > 64 >> 128 >> 5]

In [8]:
img_idx = list(range(1,30,3))
frames = len(img_idx)
image_height = 50
image_width = 50
batch_size = 1
num_classes = 5

model = Sequential()

model.add(Conv3D(16, (3,3,3), padding='same', input_shape=(frames, image_height, image_width, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

optimiser = "adam"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 10, 50, 50, 16)    1312      
                                                                 
 max_pooling3d (MaxPooling3  (None, 5, 25, 25, 16)     0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 5, 25, 25, 16)     64        
 Normalization)                                                  
                                                                 
 conv3d_1 (Conv3D)           (None, 5, 25, 25, 32)     13856     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 2, 12, 12, 32)     0         
 g3D)                                                            
                                                        

Fitting the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [9]:
train_generator = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
val_generator = generator(val_path, val_doc, batch_size, img_idx, image_height, image_width)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_path_prefix + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 1


<ipython-input-9-579c15b5d096>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Epoch 1/15
663/663 [==============================] - ETA: 0s - loss: 1.9778 - categorical_accuracy: 0.3469Source path =  /content/gdrive/My Drive/Project_data/val ; batch size = 1

Epoch 1: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00001-1.97775-0.34691-1.85271-0.39000.h5
663/663 [==============================] - 2730s 4s/step - loss: 1.9778 - categorical_accuracy: 0.3469 - val_loss: 1.8527 - val_categorical_accuracy: 0.3900 - lr: 0.0010
Epoch 2/15


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


663/663 [==============================] - ETA: 0s - loss: 1.2469 - categorical_accuracy: 0.5204
Epoch 2: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00002-1.24686-0.52036-1.59103-0.46000.h5
663/663 [==============================] - 193s 291ms/step - loss: 1.2469 - categorical_accuracy: 0.5204 - val_loss: 1.5910 - val_categorical_accuracy: 0.4600 - lr: 0.0010
Epoch 3/15
663/663 [==============================] - ETA: 0s - loss: 0.9524 - categorical_accuracy: 0.6199
Epoch 3: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00003-0.95242-0.61991-1.45746-0.52000.h5
663/663 [==============================] - 195s 294ms/step - loss: 0.9524 - categorical_accuracy: 0.6199 - val_loss: 1.4575 - val_categorical_accuracy: 0.5200 - lr: 0.0010
Epoch 4/15
663/663 [==============================] - ETA: 0s - loss: 0.6963 - categorical_accuracy: 0.7406
Epoch 4: saving model to /content/gdrive/My Drive/Models/mode

Model 2 - 100x100 - No Dropout - Layers [16 > 32 > 64 >> 128 >> 5]

In [10]:
img_idx = list(range(1,30,3))
frames = len(img_idx)
image_height = 100
image_width = 100
batch_size = 32
num_classes = 5

model = Sequential()

model.add(Conv3D(16, (3,3,3), padding='same', input_shape=(frames, image_height, image_width, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

optimiser = "adam"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 10, 100, 100, 16   1312      
                             )                                   
                                                                 
 max_pooling3d_3 (MaxPoolin  (None, 5, 50, 50, 16)     0         
 g3D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 5, 50, 50, 16)     64        
 chNormalization)                                                
                                                                 
 conv3d_4 (Conv3D)           (None, 5, 50, 50, 32)     13856     
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 2, 25, 25, 32)     0         
 g3D)                                                 

MOdel Training

In [11]:
train_generator = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
val_generator = generator(val_path, val_doc, batch_size, img_idx, image_height, image_width)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_path_prefix + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-11-579c15b5d096>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 32
Epoch 1/15
21/21 [==============================] - ETA: 0s - loss: 1.6142 - categorical_accuracy: 0.4911 Source path =  /content/gdrive/My Drive/Project_data/val ; batch size = 32

Epoch 1: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00001-1.61423-0.49107-30.87673-0.27344.h5
21/21 [==============================] - 345s 16s/step - loss: 1.6142 - categorical_accuracy: 0.4911 - val_loss: 30.8767 - val_categorical_accuracy: 0.2734 - lr: 0.0010
Epoch 2/15
21/21 [==============================] - ETA: 0s - loss: 0.5059 - categorical_accuracy: 0.8110 
Epoch 2: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00002-0.50594-0.81101-27.26493-0.31250.h5
21/21 [==============================] - 327s 16s/step - loss: 0.5059 - categorical_accuracy: 0.8110 - val_loss: 27.2649 - val_categorical_accuracy: 0.3125 - lr: 0.0010
Epoch 3/15
2

Model 3 - 100x100 - Dropout 0.5 - Layers [16 > 32 > 64 >> 128 >> 5]

In [12]:
img_idx = list(range(1,30,3))
frames = len(img_idx)
image_height = 100
image_width = 100
batch_size = 32
num_classes = 5

model = Sequential()

model.add(Conv3D(16, (3,3,3), padding='same', input_shape=(frames, image_height, image_width, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(32, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

optimiser = "adam"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_6 (Conv3D)           (None, 10, 100, 100, 16   1312      
                             )                                   
                                                                 
 max_pooling3d_6 (MaxPoolin  (None, 5, 50, 50, 16)     0         
 g3D)                                                            
                                                                 
 dropout (Dropout)           (None, 5, 50, 50, 16)     0         
                                                                 
 batch_normalization_6 (Bat  (None, 5, 50, 50, 16)     64        
 chNormalization)                                                
                                                                 
 conv3d_7 (Conv3D)           (None, 5, 50, 50, 32)     13856     
                                                      

In [13]:
train_generator = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
val_generator = generator(val_path, val_doc, batch_size, img_idx, image_height, image_width)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_path_prefix + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-13-579c15b5d096>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 32
Epoch 1/15
21/21 [==============================] - ETA: 0s - loss: 2.0192 - categorical_accuracy: 0.3557 Source path =  /content/gdrive/My Drive/Project_data/val ; batch size = 32

Epoch 1: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00001-2.01924-0.35565-16.15968-0.20312.h5
21/21 [==============================] - 328s 16s/step - loss: 2.0192 - categorical_accuracy: 0.3557 - val_loss: 16.1597 - val_categorical_accuracy: 0.2031 - lr: 0.0010
Epoch 2/15
21/21 [==============================] - ETA: 0s - loss: 1.4511 - categorical_accuracy: 0.4628 
Epoch 2: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0810_58_58.029983/model-00002-1.45109-0.46280-5.43972-0.21094.h5
21/21 [==============================] - 323s 15s/step - loss: 1.4511 - categorical_accuracy: 0.4628 - val_loss: 5.4397 - val_categorical_accuracy: 0.2109 - lr: 0.0010
Epoch 3/15
21/

Model 4 - 100x100 - Dropout 0.5 - Layers [32 > 64 > 128 >> 256 >> 5]

In [9]:
img_idx = list(range(1,30,3))
frames = len(img_idx)
image_height = 100
image_width = 100
batch_size = 64
num_classes = 5

model = Sequential()

model.add(Conv3D(32, (3,3,3), padding='same', input_shape=(frames, image_height, image_width, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(128, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

optimiser = "adam"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 10, 100, 100, 32   2624      
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 5, 50, 50, 32)     0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 5, 50, 50, 32)     0         
                                                                 
 batch_normalization (Batch  (None, 5, 50, 50, 32)     128       
 Normalization)                                                  
                                                                 
 conv3d_1 (Conv3D)           (None, 5, 50, 50, 64)     55360     
                                                        

In [10]:
train_generator = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
val_generator = generator(val_path, val_doc, batch_size, img_idx, image_height, image_width)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_path_prefix + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-10-b48246a4c71f>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 64
Epoch 1/15
11/11 [==============================] - ETA: 0s - loss: 2.3193 - categorical_accuracy: 0.3395  Source path =  /content/gdrive/My Drive/Project_data/val ; batch size = 64

Epoch 1: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0905_05_47.526708/model-00001-2.31928-0.33949-26.31317-0.23438.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


11/11 [==============================] - 2037s 200s/step - loss: 2.3193 - categorical_accuracy: 0.3395 - val_loss: 26.3132 - val_categorical_accuracy: 0.2344 - lr: 0.0010
Epoch 2/15
11/11 [==============================] - ETA: 0s - loss: 1.4331 - categorical_accuracy: 0.4815 
Epoch 2: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0905_05_47.526708/model-00002-1.43314-0.48153-33.85950-0.22656.h5
11/11 [==============================] - 565s 52s/step - loss: 1.4331 - categorical_accuracy: 0.4815 - val_loss: 33.8595 - val_categorical_accuracy: 0.2266 - lr: 0.0010
Epoch 3/15
11/11 [==============================] - ETA: 0s - loss: 1.1318 - categorical_accuracy: 0.5866 
Epoch 3: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0905_05_47.526708/model-00003-1.13179-0.58665-21.19583-0.25000.h5
11/11 [==============================] - 565s 52s/step - loss: 1.1318 - categorical_accuracy: 0.5866 - val_loss: 21.1958 - val_categorical_accuracy: 0.2500 - lr: 

Model 5 - 50x50 - Dropout 0.5 - Layers [32 > 64 > 128 >> 256 >> 5]

In [11]:
img_idx = list(range(1,30,3))
frames = len(img_idx)
image_height = 50
image_width = 50
batch_size = 64
num_classes = 5

model = Sequential()

model.add(Conv3D(32, (3,3,3), padding='same', input_shape=(frames, image_height, image_width, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Conv3D(128, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

optimiser = "adam"
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_3 (Conv3D)           (None, 10, 50, 50, 32)    2624      
                                                                 
 max_pooling3d_3 (MaxPoolin  (None, 5, 25, 25, 32)     0         
 g3D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 5, 25, 25, 32)     0         
                                                                 
 batch_normalization_3 (Bat  (None, 5, 25, 25, 32)     128       
 chNormalization)                                                
                                                                 
 conv3d_4 (Conv3D)           (None, 5, 25, 25, 64)     55360     
                                                                 
 max_pooling3d_4 (MaxPoolin  (None, 2, 12, 12, 64)    

In [12]:
train_generator = generator(train_path, train_doc, batch_size, img_idx, image_height, image_width)
val_generator = generator(val_path, val_doc, batch_size, img_idx, image_height, image_width)

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_path_prefix + model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-12-b48246a4c71f>:26: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/gdrive/My Drive/Project_data/train ; batch size = 64
Epoch 1/15
11/11 [==============================] - ETA: 0s - loss: 2.2167 - categorical_accuracy: 0.3295 Source path =  /content/gdrive/My Drive/Project_data/val ; batch size = 64

Epoch 1: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0905_05_47.526708/model-00001-2.21672-0.32955-29.89091-0.20312.h5
11/11 [==============================] - 234s 22s/step - loss: 2.2167 - categorical_accuracy: 0.3295 - val_loss: 29.8909 - val_categorical_accuracy: 0.2031 - lr: 0.0010
Epoch 2/15
11/11 [==============================] - ETA: 0s - loss: 1.7137 - categorical_accuracy: 0.4347 
Epoch 2: saving model to /content/gdrive/My Drive/Models/model_init_2024-07-0905_05_47.526708/model-00002-1.71370-0.43466-24.41684-0.21875.h5
11/11 [==============================] - 195s 18s/step - loss: 1.7137 - categorical_accuracy: 0.4347 - val_loss: 24.4168 - val_categorical_accuracy: 0.2188 - lr: 0.0010
Epoch 3/15
1